In [1]:

import numpy as np
import pandas as pd 
from sklearn.cluster import KMeans
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
from bs4 import BeautifulSoup
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
import folium
import requests
from pandas.io.json import json_normalize
print("Libraries imported.")

Libraries imported.


In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
data

'\n<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XptJpgpAICsAAFpdq2oAAAAK","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":951325562,"wgRevisionId":951325562,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related l

In [3]:
soup = BeautifulSoup(data, 'html.parser')

In [4]:
postalcodes = []
borough = []
neighborhood = []

soup.find('table').find_all('tr')

soup.find('table').find_all('tr')
for row in soup.find('table').find_all('tr'):
    cel = row.find_all('td')


In [5]:
for row in soup.find('table').find_all('tr'):
    cel= row.find_all('td')
    if(len(cel) > 0):
        postalcodes.append(cel[0].text)
        borough.append(cel[1].text)
        neighborhood.append(cel[2].text.rstrip('\n')) 

In [6]:
toronto = pd.DataFrame({"PostalCode": postalcodes,
                           "Borough": borough,
                           "Neighborhood": neighborhood})

In [7]:
toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront


In [8]:
toronto_data = toronto[toronto['Borough'] != "Not assigned\n"].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront
3,M6A\n,North York\n,Lawrence Manor / Lawrence Heights
4,M7A\n,Downtown Toronto\n,Queen's Park / Ontario Provincial Government


In [9]:
toronto_group = toronto_data.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_group=toronto_group[toronto_group['Borough']!="Not assigned"].reset_index(drop=True)
toronto_group.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,Malvern / Rouge
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [10]:
for index, row in toronto_group.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_group.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,Malvern / Rouge
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [11]:
for index, row in toronto_group.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_group.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,Malvern / Rouge
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [12]:
toronto_group.shape

(103, 3)

In [13]:
loc=pd.read_csv('/home/lenovo/Downloads/Geospatial_Coordinates.csv')
loc.head()
loc.shape

(103, 3)

In [14]:
loc.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
loc.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
new_toronto = toronto_group.merge(loc, on="PostalCode", how="left")
new_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B\n,Scarborough\n,Malvern / Rouge,NaN,NaN
1,M1C\n,Scarborough\n,Rouge Hill / Port Union / Highland Creek,NaN,NaN
2,M1E\n,Scarborough\n,Guildwood / Morningside / West Hill,NaN,NaN
3,M1G\n,Scarborough\n,Woburn,NaN,NaN
4,M1H\n,Scarborough\n,Cedarbrae,NaN,NaN


In [16]:
new_toronto.shape

(103, 5)

In [17]:
new_toronto = new_toronto.replace('\n','', regex=True)
new_toronto=new_toronto.replace('/',',',regex=True)
new_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",NaN,NaN
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",NaN,NaN
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,NaN,NaN
4,M1H,Scarborough,Cedarbrae,NaN,NaN


In [18]:

new_toronto=new_toronto.replace('/',',',regex=True)
#new_toronto.head()


In [19]:
new_toronto['Latitude']= ""
new_toronto['Longitude']= ""
for i,h in loc.iterrows():
    new_toronto['Latitude'].loc[i]=h[1]
    new_toronto['Longitude'].loc[i]=h[2]
new_toronto.head()
#new_toronto.drop('latitude',1,inplace=True)
    

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.8067,-79.1944
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


In [20]:
toronto_df = new_toronto.merge(loc, on="PostalCode", how="left")
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M1B,Scarborough,"Malvern , Rouge",43.8067,-79.1944,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.7845,-79.1605,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.7636,-79.1887,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.771,-79.2169,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395,43.773136,-79.239476


In [21]:
#toronto_df.drop(['Latitude_x','Longitude_x'],1,inplace=True)
toronto_df.rename(columns={'Latitude_y':'Latitude','Longitude_y':'Longitude'},inplace=True)
toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


In [22]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [23]:
map_ = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_)  
    
map_

In [25]:
borough_names = list(toronto_df.Borough.unique())

toronto_borough = []

for x in borough_names:
    if "toronto" in x.lower():
        toronto_borough.append(x)
        
toronto_borough

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [37]:
new_df=toronto_df[toronto_df['Borough'].isin(toronto_borough)].reset_index(drop=True)
new_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [45]:
toronto=folium.Map(location=[latitude,longitude],zoom_start=12)
for lat,lng,borough in zip(new_df['Latitude'],new_df['Longitude'],new_df['Borough']):
    label=folium.Popup(borough,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
         fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto) 
        
    
    
toronto   

In [46]:
CLIENT_ID = '1RADDPZKPBM32VEDMKLVBUUJJUHZZFK5F3DEMVR5H4IUJZDO' # your Foursquare ID
CLIENT_SECRET = 'KPADLA0CICG0OZIRZI513I34VM51A2OX1FA2GOMOFKE2PWZR'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1RADDPZKPBM32VEDMKLVBUUJJUHZZFK5F3DEMVR5H4IUJZDO
CLIENT_SECRET:KPADLA0CICG0OZIRZI513I34VM51A2OX1FA2GOMOFKE2PWZR


In [49]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['PostalCode'], new_df['Borough'], 
                                                  new_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ&client_secret=OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H&v=20180605 \
     &ll=43.653963,-79.387207&radius=500&limit=100".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [53]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1677, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Japango,43.655268,-79.385165,Sushi Restaurant
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Rolltation,43.654918,-79.387424,Japanese Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Karine's,43.653699,-79.390743,Breakfast Spot


In [91]:

t_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

t_onehot['PostalCode'] = venues_df['PostalCode'] 
t_onehot['Borough'] = venues_df['Borough'] 
t_onehot['Neighborhoods'] = venues_df['Neighborhood'] 
columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
t_onehot = t_onehot[columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1677, 35)


,PostalCode,Borough,Neighborhoods,American Restaurant,Art Gallery,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,Café,Coffee Shop,Concert Hall,Donut Shop,French Restaurant,Gastropub,Gift Shop,Hotel,Japanese Restaurant,Juice Bar,Korean Restaurant,Modern European Restaurant,Movie Theater,Neighborhood,Opera House,Park,Plaza,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,Tapas Restaurant,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,M4E,East Toronto,The Beaches,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
toronto_grouped= toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 35)


,PostalCode,Borough,Neighborhoods,American Restaurant,Art Gallery,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,Café,Coffee Shop,Concert Hall,Donut Shop,French Restaurant,Gastropub,Gift Shop,Hotel,Japanese Restaurant,Juice Bar,Korean Restaurant,Modern European Restaurant,Movie Theater,Neighborhood,Opera House,Park,Plaza,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,Tapas Restaurant,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0.023256,0.069767,0.023256,0.023256,0.046512,0.023256,0.069767,0.093023,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
1,M4K,East Toronto,"The Danforth West , Riverdale",0.023256,0.069767,0.023256,0.023256,0.046512,0.023256,0.069767,0.093023,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
2,M4L,East Toronto,"India Bazaar , The Beaches West",0.023256,0.069767,0.023256,0.023256,0.046512,0.023256,0.069767,0.093023,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
3,M4M,East Toronto,Studio District,0.023256,0.069767,0.023256,0.023256,0.046512,0.023256,0.069767,0.093023,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
4,M4N,Central Toronto,Lawrence Park,0.023256,0.069767,0.023256,0.023256,0.046512,0.023256,0.069767,0.093023,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
5,M4P,Central Toronto,Davisville North,0.023256,0.069767,0.023256,0.023256,0.046512,0.023256,0.069767,0.093023,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
6,M4R,Central Toronto,North Toronto West,0.023256,0.069767,0.023256,0.023256,0.046512,0.023256,0.069767,0.093023,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
7,M4S,Central Toronto,Davisville,0.023256,0.069767,0.023256,0.023256,0.046512,0.023256,0.069767,0.093023,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
8,M4T,Central Toronto,"Moore Park , Summerhill East",0.023256,0.069767,0.023256,0.023256,0.046512,0.023256,0.069767,0.093023,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
9,M4V,Central Toronto,"Summerhill West , Rathnelly , South Hill , For...",0.023256,0.069767,0.023256,0.023256,0.046512,0.023256,0.069767,0.093023,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.069767,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256


In [92]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop
1,M4K,East Toronto,"The Danforth West , Riverdale",Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop
2,M4L,East Toronto,"India Bazaar , The Beaches West",Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop
3,M4M,East Toronto,Studio District,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop
4,M4N,Central Toronto,Lawrence Park,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop
5,M4P,Central Toronto,Davisville North,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop
6,M4R,Central Toronto,North Toronto West,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop
7,M4S,Central Toronto,Davisville,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop
8,M4T,Central Toronto,"Moore Park , Summerhill East",Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop
9,M4V,Central Toronto,"Summerhill West , Rathnelly , South Hill , For...",Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop


In [93]:
kclusters = 3
toronto_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=4).fit(toronto_clustering)

kmeans.labels_[0:10]


/home/lenovo/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [94]:
toronto_merged = new_df.copy()

toronto_merged["Cluster Labels"] = kmeans.labels_

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head()

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Coffee Shop,Japanese Restaurant,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Bar,Bubble Tea Shop


In [95]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
